# Calling AWS bedrock models

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## 1 Boto3 version

### 1.1 Non Stream

In [ ]:
import boto3
import json

# Initialize a session using Amazon Bedrock
session = boto3.Session(region_name='us-east-1')

# Create a Bedrock runtime client
bedrock_client = session.client('bedrock-runtime')

# Prepare the payload
payload = {
    "inputText": "Hi",
    "textGenerationConfig": {
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}

# Invoke the model
response = bedrock_client.invoke_model(
    modelId='amazon.titan-text-lite-v1',
    contentType='application/json',
    accept='application/json',
    body=json.dumps(payload)
)

# Read and print the response
response_body = response['body'].read().decode('utf-8')
print(response_body)


### 1.2 Stream

In [ ]:
import boto3
import json

# Initialize a session using Amazon Bedrock
session = boto3.Session(region_name='us-east-1')

# Create a Bedrock runtime client
bedrock_client = session.client('bedrock-runtime')

# Prepare the payload
payload = {
    "inputText": "Write 3 paragraphs about dinassaurs.",
    "textGenerationConfig": {
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}

# Invoke the model with streaming response
response = bedrock_client.invoke_model_with_response_stream(
    modelId='amazon.titan-text-lite-v1',
    contentType='application/json',
    accept='application/json',
    body=json.dumps(payload)
)

stream = response.get('body')
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            print(json.loads(chunk.get('bytes').decode()))


# 2. Raw calls

### 2.1 NonStream

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import sys
import os
import datetime
import hashlib
import hmac
import requests
import json
from dotenv import load_dotenv
load_dotenv()

# ************* CONFIGURATION *************
# AWS credentials
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

if access_key is None or secret_key is None:
    print('Error: AWS credentials are not set in environment variables.')
    sys.exit(1)

# Service and region
service = 'bedrock'
region = 'us-east-1'

# Host and endpoint
host = f'bedrock-runtime.{region}.amazonaws.com'
endpoint = f'https://{host}/model/amazon.titan-text-lite-v1/invoke'

# Request parameters
payload = {
    "inputText": "Hi",
    "textGenerationConfig": {
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}

request_parameters = json.dumps(payload)
content_type = 'application/json'

# ************* HELPER FUNCTIONS *************
def sign(key, msg):
    return hmac.new(key, msg.encode('utf-8'), hashlib.sha256).digest()

def get_signature_key(key, date_stamp, region_name, service_name):
    k_date = sign(('AWS4' + key).encode('utf-8'), date_stamp)
    k_region = sign(k_date, region_name)
    k_service = sign(k_region, service_name)
    k_signing = sign(k_service, 'aws4_request')
    return k_signing

# ************* TASK 1: CREATE A CANONICAL REQUEST *************
def create_canonical_request(method, uri, query_string, canonical_headers, signed_headers, payload_hash):
    canonical_request = '\n'.join([
        method,
        uri,
        query_string,
        canonical_headers,
        signed_headers,
        payload_hash
    ])
    return canonical_request

# ************* TASK 2: CREATE THE STRING TO SIGN *************
def create_string_to_sign(algorithm, amz_date, credential_scope, canonical_request):
    string_to_sign = '\n'.join([
        algorithm,
        amz_date,
        credential_scope,
        hashlib.sha256(canonical_request.encode('utf-8')).hexdigest()
    ])
    return string_to_sign

# ************* TASK 3: CALCULATE THE SIGNATURE *************
def calculate_signature(secret_key, date_stamp, region, service, string_to_sign):
    signing_key = get_signature_key(secret_key, date_stamp, region, service)
    signature = hmac.new(signing_key, string_to_sign.encode('utf-8'), hashlib.sha256).hexdigest()
    return signature

# ************* TASK 4: ADD SIGNING INFORMATION TO THE REQUEST *************
def create_authorization_header(access_key, credential_scope, signed_headers, signature):
    authorization_header = (
        f"AWS4-HMAC-SHA256 Credential={access_key}/{credential_scope}, "
        f"SignedHeaders={signed_headers}, Signature={signature}"
    )
    return authorization_header

# ************* MAIN FUNCTION *************
def main():
    method = 'POST'
    uri = '/model/amazon.titan-text-lite-v1/invoke'
    query_string = ''

    # Timestamp and date
    t = datetime.datetime.utcnow()
    amz_date = t.strftime('%Y%m%dT%H%M%SZ')  # Format: YYYYMMDD'T'HHMMSS'Z'
    date_stamp = t.strftime('%Y%m%d')  # Date without time for credential scope

    # Payload hash
    payload_hash = hashlib.sha256(request_parameters.encode('utf-8')).hexdigest()

    # Headers (lowercase)
    headers = {
        'content-type': content_type,
        'host': host,
        'x-amz-date': amz_date,
        'x-amz-content-sha256': payload_hash
    }

    # Canonical headers and signed headers
    sorted_header_keys = sorted(headers.keys())
    canonical_headers = ''
    signed_headers = ''
    for key in sorted_header_keys:
        canonical_headers += f"{key}:{headers[key]}\n"
        signed_headers += f"{key};"
    signed_headers = signed_headers.rstrip(';')

    # Create canonical request
    canonical_request = create_canonical_request(
        method,
        uri,
        query_string,
        canonical_headers,
        signed_headers,
        payload_hash
    )

    # Algorithm and credential scope
    algorithm = 'AWS4-HMAC-SHA256'
    credential_scope = f"{date_stamp}/{region}/{service}/aws4_request"

    # Create string to sign
    string_to_sign = create_string_to_sign(
        algorithm,
        amz_date,
        credential_scope,
        canonical_request
    )

    # Calculate the signature
    signature = calculate_signature(
        secret_key,
        date_stamp,
        region,
        service,
        string_to_sign
    )

    # Create authorization header
    authorization_header = create_authorization_header(
        access_key,
        credential_scope,
        signed_headers,
        signature
    )

    # Add authorization header to headers
    headers['Authorization'] = authorization_header

    # Remove 'host' from the headers for the actual request (requests library adds it automatically)
    request_headers = {k: v for k, v in headers.items() if k != 'host'}

    # Make the request
    print('Making request to AWS Bedrock...')
    response = requests.post(endpoint, data=request_parameters, headers=request_headers)
    print(response)

    # Check the response
    print(f"Response Code: {response.status_code}")
    print("Response Body:")
    print(response.text)

if __name__ == '__main__':
    main()


### 2.2 Stream

In [ ]:
import sys
import os
import datetime
import hashlib
import hmac
import requests
import json
import base64
import struct
import binascii
import io

# ************* CONFIGURATION *************
# AWS credentials
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

if access_key is None or secret_key is None:
    print('Error: AWS credentials are not set in environment variables.')
    sys.exit(1)

# Service and region
service = 'bedrock'
region = 'us-east-1'

# Host and endpoint
host = f'bedrock-runtime.{region}.amazonaws.com'
# Endpoint path
uri = '/model/amazon.titan-text-lite-v1/invoke-with-response-stream'
endpoint = f'https://{host}{uri}'

# Request parameters
payload = {
    "inputText": "Write 3 paragraphs about dinossaurs.",
    "textGenerationConfig": {
        "maxTokenCount": 4096,
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}
request_parameters = json.dumps(payload)
content_type = 'application/json'
accept = 'application/json'  # Use 'application/json' for EventStream format

# ************* HELPER FUNCTIONS *************
def sign(key, msg):
    return hmac.new(key, msg.encode('utf-8'), hashlib.sha256).digest()

def get_signature_key(key, date_stamp, region_name, service_name):
    k_date = sign(('AWS4' + key).encode('utf-8'), date_stamp)
    k_region = sign(k_date, region_name)
    k_service = sign(k_region, service_name)
    k_signing = sign(k_service, 'aws4_request')
    return k_signing

# ************* TASK 1: CREATE A CANONICAL REQUEST *************
def create_canonical_request(method, uri, query_string, canonical_headers, signed_headers, payload_hash):
    canonical_request = '\n'.join([
        method,
        uri,
        query_string,
        canonical_headers,
        signed_headers,
        payload_hash
    ])
    return canonical_request

# ************* TASK 2: CREATE THE STRING TO SIGN *************
def create_string_to_sign(algorithm, amz_date, credential_scope, canonical_request):
    string_to_sign = '\n'.join([
        algorithm,
        amz_date,
        credential_scope,
        hashlib.sha256(canonical_request.encode('utf-8')).hexdigest()
    ])
    return string_to_sign

# ************* TASK 3: CALCULATE THE SIGNATURE *************
def calculate_signature(secret_key, date_stamp, region, service, string_to_sign):
    signing_key = get_signature_key(secret_key, date_stamp, region, service)
    signature = hmac.new(signing_key, string_to_sign.encode('utf-8'), hashlib.sha256).hexdigest()
    return signature

# ************* TASK 4: ADD SIGNING INFORMATION TO THE REQUEST *************
def create_authorization_header(access_key, credential_scope, signed_headers, signature):
    authorization_header = (
        f"AWS4-HMAC-SHA256 Credential={access_key}/{credential_scope}, "
        f"SignedHeaders={signed_headers}, Signature={signature}"
    )
    return authorization_header

# ************* EVENTSTREAM PARSER FUNCTION *************
def parse_event_stream_incremental(buffer):
    buffer_length = len(buffer)
    offset = 0
    while True:
        if buffer_length - offset < 4:
            # Not enough data to read total_length
            break
        total_length = struct.unpack('>I', buffer[offset:offset+4])[0]
        if buffer_length - offset < total_length:
            # Not enough data to read the whole message
            break
        message = buffer[offset:offset+total_length]
        # Parse the prelude
        headers_length = struct.unpack('>I', message[4:8])[0]
        prelude_crc = message[8:12]
        prelude = message[:8]
        computed_prelude_crc = binascii.crc32(prelude) & 0xffffffff
        if struct.unpack('>I', prelude_crc)[0] != computed_prelude_crc:
            raise ValueError('Prelude CRC mismatch')

        # Parse headers
        headers = {}
        pos = 12  # Starting position after prelude and prelude CRC
        headers_end = pos + headers_length
        if headers_end > total_length - 4:
            raise ValueError('Headers extend beyond message length')
        while pos < headers_end:
            if pos + 1 > total_length:
                raise ValueError('Incomplete header')
            name_len = message[pos]
            pos += 1
            if pos + name_len > total_length:
                raise ValueError('Incomplete header name')
            name = message[pos:pos+name_len].decode('utf-8')
            pos += name_len
            if pos + 1 > total_length:
                raise ValueError('Incomplete value type')
            value_type = message[pos]
            pos += 1
            if value_type == 7:  # String
                if pos + 2 > total_length:
                    raise ValueError('Incomplete value length')
                value_len = struct.unpack('>H', message[pos:pos+2])[0]
                pos += 2
                if pos + value_len > total_length:
                    raise ValueError('Incomplete value')
                value = message[pos:pos+value_len].decode('utf-8')
                pos += value_len
            else:
                # Handle other value types if necessary
                value = None
            headers[name] = value

        # Payload
        payload = message[headers_end:-4]  # Exclude the Message CRC at the end

        # Message CRC
        message_crc = struct.unpack('>I', message[-4:])[0]
        computed_message_crc = binascii.crc32(message[:-4]) & 0xffffffff
        if message_crc != computed_message_crc:
            raise ValueError('Message CRC mismatch')

        # Process the event
        message_length = total_length
        yield headers, payload, message_length

        offset += total_length

        if offset >= buffer_length:
            break

# ************* MAIN FUNCTION *************
def main():
    method = 'POST'
    query_string = ''

    # Timestamp and date
    t = datetime.datetime.now(datetime.UTC)
    amz_date = t.strftime('%Y%m%dT%H%M%SZ')  # Format: YYYYMMDD'T'HHMMSS'Z'
    date_stamp = t.strftime('%Y%m%d')  # Date without time for credential scope

    # Payload hash
    payload_hash = hashlib.sha256(request_parameters.encode('utf-8')).hexdigest()

    # Headers (lowercase)
    headers = {
        'content-type': content_type,
        'host': host,
        'x-amz-date': amz_date,
        'x-amz-content-sha256': payload_hash,
        'accept': accept
    }

    # Canonical headers and signed headers
    sorted_header_keys = sorted(headers.keys())
    canonical_headers = ''
    signed_headers = ''
    for key in sorted_header_keys:
        canonical_headers += f"{key}:{headers[key]}\n"
        signed_headers += f"{key};"
    signed_headers = signed_headers.rstrip(';')

    # Create canonical request
    canonical_request = create_canonical_request(
        method,
        uri,
        query_string,
        canonical_headers,
        signed_headers,
        payload_hash
    )

    # Algorithm and credential scope
    algorithm = 'AWS4-HMAC-SHA256'
    credential_scope = f"{date_stamp}/{region}/{service}/aws4_request"

    # Create string to sign
    string_to_sign = create_string_to_sign(
        algorithm,
        amz_date,
        credential_scope,
        canonical_request
    )

    # Calculate the signature
    signature = calculate_signature(
        secret_key,
        date_stamp,
        region,
        service,
        string_to_sign
    )

    # Create authorization header
    authorization_header = create_authorization_header(
        access_key,
        credential_scope,
        signed_headers,
        signature
    )

    # Add authorization header to headers
    headers['Authorization'] = authorization_header

    # Remove 'host' from the headers for the actual request (requests library adds it automatically)
    request_headers = {k: v for k, v in headers.items() if k.lower() != 'host'}

    # Make the request with streaming enabled
    print('Making streaming request to AWS Bedrock...')
    response = requests.post(endpoint, data=request_parameters, headers=request_headers, stream=True)

    # Check the response status
    print(f"Response Code: {response.status_code}")

    if response.status_code != 200:
        print("Error in response:")
        print(response.text)
        return

    # Process the streaming response
    print("Streaming response:")

    buffer = b''  # Initialize an empty buffer

    for chunk in response.iter_content(chunk_size=None):
        if chunk:
            buffer += chunk  # Add the chunk to the buffer

            while True:
                try:
                    parsed = False
                    for headers, payload, message_length in parse_event_stream_incremental(buffer):
                        parsed = True
                        # Process the message
                        event_type = headers.get(':event-type')
                        if event_type == 'chunk':
                            # The payload is JSON
                            data = json.loads(payload)
                            # The 'bytes' field is base64-encoded
                            decoded_bytes = base64.b64decode(data['bytes'])
                            text = decoded_bytes.decode('utf-8')
                            print(text)  # Print the text as it comes
                        elif event_type == 'error':
                            # Handle errors
                            error_code = headers.get(':error-code')
                            error_message = headers.get(':error-message')
                            print(f"\nError {error_code}: {error_message}")
                            return
                        # Remove the parsed message from the buffer
                        buffer = buffer[message_length:]
                        break  # Process next message
                    if not parsed:
                        # No complete message could be parsed
                        break
                except ValueError as e:
                    # Incomplete message or parsing error
                    break  # Wait for more data

if __name__ == '__main__':
    main()


# 3. Converse API - Stream

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import sys
import os
import datetime
import hashlib
import hmac
import requests
import json
import base64
import struct
import binascii

# ************* CONFIGURATION *************
# AWS credentials
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

if access_key is None or secret_key is None:
    print('Error: AWS credentials are not set in environment variables.')
    sys.exit(1)

# Service and region
service = 'bedrock'
region = 'us-east-1'

# Host and endpoint
host = f'bedrock-runtime.{region}.amazonaws.com'
# Update the URI to use converse-stream
uri = '/model/amazon.titan-text-lite-v1/converse-stream'
endpoint = f'https://{host}{uri}'

# Request parameters for ConverseStream
payload = {
    "messages": [
        {
            "content": [
                {
                    "text": "Hello how are you? Can you write 3 paragraphs about dinossaurs? Make them about anything related to them!"
                }
            ],
            "role": "user"
        }
    ],
    "system": None,
    "inferenceConfig": {
        "maxTokens": 150,  # Adjust as needed
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}
request_parameters = json.dumps(payload)
content_type = 'application/json'
accept = 'application/json'

# ************* HELPER FUNCTIONS *************
def sign(key, msg):
    return hmac.new(key, msg.encode('utf-8'), hashlib.sha256).digest()

def get_signature_key(key, date_stamp, region_name, service_name):
    k_date = sign(('AWS4' + key).encode('utf-8'), date_stamp)
    k_region = sign(k_date, region_name)
    k_service = sign(k_region, service_name)
    k_signing = sign(k_service, 'aws4_request')
    return k_signing

# ************* EVENT STREAM PARSING FUNCTIONS *************
def parse_event_stream(buffer):
    messages = []
    offset = 0
    while offset < len(buffer):
        if len(buffer) - offset < 4:
            # Not enough data to read total_length
            break
        total_length = struct.unpack('>I', buffer[offset:offset+4])[0]
        if len(buffer) - offset < total_length:
            # Not enough data to read the whole message
            break
        message = buffer[offset:offset+total_length]
        headers, payload = parse_event_stream_message(message)
        messages.append((headers, payload))
        offset += total_length
    return messages, buffer[offset:]

def parse_event_stream_message(message):
    # Parse the prelude
    headers_length = struct.unpack('>I', message[4:8])[0]
    prelude_crc = struct.unpack('>I', message[8:12])[0]
    prelude = message[:8]
    computed_prelude_crc = binascii.crc32(prelude) & 0xffffffff
    if prelude_crc != computed_prelude_crc:
        raise ValueError('Prelude CRC mismatch')

    # Parse headers
    headers = {}
    pos = 12  # Starting position after prelude and prelude CRC
    headers_end = pos + headers_length
    while pos < headers_end:
        name_len = message[pos]
        pos += 1
        name = message[pos:pos+name_len].decode('utf-8')
        pos += name_len
        value_type = message[pos]
        pos += 1
        if value_type == 7:  # String
            value_len = struct.unpack('>H', message[pos:pos+2])[0]
            pos += 2
            value = message[pos:pos+value_len].decode('utf-8')
            pos += value_len
        else:
            # Handle other value types if necessary
            value = None
        headers[name] = value

    # Payload
    payload = message[headers_end:-4]  # Exclude the Message CRC at the end

    # Verify message CRC
    message_crc = struct.unpack('>I', message[-4:])[0]
    computed_message_crc = binascii.crc32(message[:-4]) & 0xffffffff
    if message_crc != computed_message_crc:
        raise ValueError('Message CRC mismatch')

    return headers, payload

# ************* MAIN FUNCTION *************
def main():
    method = 'POST'
    query_string = ''

    # Timestamp and date
    t = datetime.datetime.now(datetime.UTC)
    amz_date = t.strftime('%Y%m%dT%H%M%SZ')  # Format: YYYYMMDD'T'HHMMSS'Z'
    date_stamp = t.strftime('%Y%m%d')  # Date without time for credential scope

    # Payload hash
    payload_hash = hashlib.sha256(request_parameters.encode('utf-8')).hexdigest()

    # Headers (lowercase)
    headers = {
        'content-type': content_type,
        'host': host,
        'x-amz-date': amz_date,
        'x-amz-content-sha256': payload_hash,
        'accept': accept
    }

    # Canonical headers and signed headers
    sorted_header_keys = sorted(headers.keys())
    canonical_headers = ''
    signed_headers = ''
    for key in sorted_header_keys:
        canonical_headers += f"{key}:{headers[key]}\n"
        signed_headers += f"{key};"
    signed_headers = signed_headers.rstrip(';')

    # Create canonical request
    canonical_request = '\n'.join([
        method,
        uri,
        query_string,
        canonical_headers,
        signed_headers,
        payload_hash
    ])

    # Algorithm and credential scope
    algorithm = 'AWS4-HMAC-SHA256'
    credential_scope = f"{date_stamp}/{region}/{service}/aws4_request"

    # Create string to signx
    string_to_sign = '\n'.join([
        algorithm,
        amz_date,
        credential_scope,
        hashlib.sha256(canonical_request.encode('utf-8')).hexdigest()
    ])

    # Calculate the signature
    signing_key = get_signature_key(secret_key, date_stamp, region, service)
    signature = hmac.new(signing_key, string_to_sign.encode('utf-8'), hashlib.sha256).hexdigest()

    # Create authorization header
    authorization_header = (
        f"{algorithm} Credential={access_key}/{credential_scope}, "
        f"SignedHeaders={signed_headers}, Signature={signature}"
    )

    # Add authorization header to headers
    headers['Authorization'] = authorization_header

    # Remove 'host' from the headers for the actual request (requests library adds it automatically)
    request_headers = {k: v for k, v in headers.items() if k.lower() != 'host'}

    # Make the request with streaming enabled
    print('Making streaming request to AWS Bedrock...')
    response = requests.post(endpoint, data=request_parameters, headers=request_headers, stream=True)

    # Check the response status
    print(f"Response Code: {response.status_code}")

    if response.status_code != 200:
        print("Error in response:")
        print(response.text)
        return

    # Process the streaming response
    print("Streaming response:")

    buffer = b''  # Initialize an empty buffer
    chunk_count = 0
    for chunk in response.iter_content(chunk_size=None):
        if chunk:
            
            print(f'chunk_count: {chunk_count}')
            chunk_count += 1
            buffer += chunk  # Add the chunk to the buffer

            while True:
                messages, buffer = parse_event_stream(buffer)
                if not messages:
                    break  # Wait for more data
                for headers, payload in messages:
                    # Process the message
                    event_type = headers.get(':event-type')
                    if event_type == 'messageStart':
                        # Start of a new message
                        pass
                    elif event_type == 'contentBlockDelta':
                        data = json.loads(payload.decode('utf-8'))
                        delta = data.get('delta', {})
                        text = delta.get('text', '')
                        print(text)
                    elif event_type == 'contentBlockStop':
                        # End of a content block
                        pass
                    elif event_type == 'messageStop':
                        # End of a message
                        pass
                    elif event_type == 'metadata':
                        # Process metadata if needed
                        data = json.loads(payload.decode('utf-8'))
                        # For example, print usage metrics
                        usage = data.get('usage', {})
                        if usage:
                            print(f"\nUsage: {usage}")
                    elif event_type == 'error':
                        # Handle errors
                        error_code = headers.get(':error-code')
                        error_message = headers.get(':error-message')
                        print(f"\nError {error_code}: {error_message}")
                        return
                    else:
                        # Handle other event types
                        pass

if __name__ == '__main__':
    main()


In [ ]:
import sys
import os
import datetime
import hashlib
import hmac
import requests
import json
import base64
import struct
import binascii

# ************* CONFIGURATION *************
# AWS credentials
access_key = os.environ.get('AWS_ACCESS_KEY_ID')
secret_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

if access_key is None or secret_key is None:
    print('Error: AWS credentials are not set in environment variables.')
    sys.exit(1)

# Service and region
service = 'bedrock'
region = 'us-east-1'

# Host and endpoint
host = f'bedrock-runtime.{region}.amazonaws.com'
# Update the URI to use converse-stream
uri = '/model/amazon.titan-text-lite-v1/converse-stream'
endpoint = f'https://{host}{uri}'

# Request parameters for ConverseStream
payload = {
    "messages": [
        {
            "content": [
                {
                    "text": "Hello how are you? Can you write 3 paragraphs about dinosaurs? Make them about anything related to them!"
                }
            ],
            "role": "user"
        }
    ],
    "system": None,
    "inferenceConfig": {
        "maxTokens": 150,  # Adjust as needed
        "stopSequences": [],
        "temperature": 0,
        "topP": 0.9
    }
}
request_parameters = json.dumps(payload)
content_type = 'application/json'
accept = 'application/json'

# ************* HELPER FUNCTIONS *************
def sign(key, msg):
    return hmac.new(key, msg.encode('utf-8'), hashlib.sha256).digest()

def get_signature_key(key, date_stamp, region_name, service_name):
    k_date = sign(('AWS4' + key).encode('utf-8'), date_stamp)
    k_region = sign(k_date, region_name)
    k_service = sign(k_region, service_name)
    k_signing = sign(k_service, 'aws4_request')
    return k_signing

# ************* EVENT STREAM PARSING FUNCTIONS *************
def parse_event_stream(buffer):
    messages = []
    offset = 0
    while offset + 12 <= len(buffer):
        total_length = struct.unpack('>I', buffer[offset:offset+4])[0]
        if offset + total_length > len(buffer):
            # Not enough data to read the whole message
            break
        message = buffer[offset:offset+total_length]
        headers, payload = parse_event_stream_message(message)
        messages.append((headers, payload))
        offset += total_length
    remaining_buffer = buffer[offset:]
    return messages, remaining_buffer

def parse_event_stream_message(message):
    total_length = struct.unpack('>I', message[0:4])[0]
    headers_length = struct.unpack('>I', message[4:8])[0]
    prelude_crc = struct.unpack('>I', message[8:12])[0]
    prelude = message[0:8]
    computed_prelude_crc = binascii.crc32(prelude) & 0xffffffff
    if prelude_crc != computed_prelude_crc:
        raise ValueError('Prelude CRC mismatch')

    # Parse headers
    headers = {}
    pos = 12  # Starting position after prelude and prelude CRC
    headers_end = pos + headers_length
    while pos < headers_end:
        name_len = message[pos]
        pos += 1
        name = message[pos:pos+name_len].decode('utf-8')
        pos += name_len
        value_type = message[pos]
        pos += 1
        if value_type == 7:  # String
            value_len = struct.unpack('>H', message[pos:pos+2])[0]
            pos += 2
            value = message[pos:pos+value_len].decode('utf-8')
            pos += value_len
        else:
            # Handle other value types if necessary
            value = None
        headers[name] = value

    # Payload
    payload = message[headers_end:-4]  # Exclude the Message CRC at the end

    # Verify message CRC
    message_crc = struct.unpack('>I', message[-4:])[0]
    computed_message_crc = binascii.crc32(message[:-4]) & 0xffffffff
    if message_crc != computed_message_crc:
        raise ValueError('Message CRC mismatch')

    return headers, payload

# New function to parse chunk and return headers and payload as strings
def parse_chunk(chunk, buffer):
    buffer += chunk  # Add the new chunk to the buffer
    messages = []
    while True:
        parsed_messages, buffer = parse_event_stream(buffer)
        if not parsed_messages:
            break  # Wait for more data
        messages.extend(parsed_messages)
    return messages, buffer

# ************* MAIN FUNCTION *************
def main():
    method = 'POST'
    query_string = ''

    # Timestamp and date
    t = datetime.datetime.utcnow()
    amz_date = t.strftime('%Y%m%dT%H%M%SZ')  # Format: YYYYMMDD'T'HHMMSS'Z'
    date_stamp = t.strftime('%Y%m%d')  # Date without time for credential scope

    # Payload hash
    payload_hash = hashlib.sha256(request_parameters.encode('utf-8')).hexdigest()

    # Headers (lowercase)
    headers = {
        'content-type': content_type,
        'host': host,
        'x-amz-date': amz_date,
        'x-amz-content-sha256': payload_hash,
        'accept': accept
    }

    # Canonical headers and signed headers
    sorted_header_keys = sorted(headers.keys())
    canonical_headers = ''
    signed_headers = ''
    for key in sorted_header_keys:
        canonical_headers += f"{key}:{headers[key]}\n"
        signed_headers += f"{key};"
    signed_headers = signed_headers.rstrip(';')

    # Create canonical request
    canonical_request = '\n'.join([
        method,
        uri,
        query_string,
        canonical_headers,
        signed_headers,
        payload_hash
    ])

    # Algorithm and credential scope
    algorithm = 'AWS4-HMAC-SHA256'
    credential_scope = f"{date_stamp}/{region}/{service}/aws4_request"

    # Create string to sign
    string_to_sign = '\n'.join([
        algorithm,
        amz_date,
        credential_scope,
        hashlib.sha256(canonical_request.encode('utf-8')).hexdigest()
    ])

    # Calculate the signature
    signing_key = get_signature_key(secret_key, date_stamp, region, service)
    signature = hmac.new(signing_key, string_to_sign.encode('utf-8'), hashlib.sha256).hexdigest()

    # Create authorization header
    authorization_header = (
        f"{algorithm} Credential={access_key}/{credential_scope}, "
        f"SignedHeaders={signed_headers}, Signature={signature}"
    )

    # Add authorization header to headers
    headers['Authorization'] = authorization_header

    # Remove 'host' from the headers for the actual request (requests library adds it automatically)
    request_headers = {k: v for k, v in headers.items() if k.lower() != 'host'}

    # Make the request with streaming enabled
    print('Making streaming request to AWS Bedrock...')
    response = requests.post(endpoint, data=request_parameters, headers=request_headers, stream=True)

    # Check the response status
    print(f"Response Code: {response.status_code}")

    if response.status_code != 200:
        print("Error in response:")
        print(response.text)
        return

    # Process the streaming response
    print("Streaming response:")

    buffer = b''  # Initialize an empty buffer
    chunk_count = 0
    for chunk in response.iter_content(chunk_size=None):
        if chunk:
            print(f'chunk_count: {chunk_count}')
            chunk_count += 1

            # Parse the chunk using the new function
            messages, buffer = parse_chunk(chunk, buffer)
            for headers, payload in messages:
                # Convert headers and payload to strings
                headers_str = json.dumps(headers)
                payload_str = payload.decode('utf-8')

                # Now you can use headers_str and payload_str as needed
                print(f"Headers: {headers_str}")
                print(f"Payload: {payload_str}")

                # Process the message based on event type
                event_type = headers.get(':event-type')
                if event_type == 'messageStart':
                    # Start of a new message
                    pass
                elif event_type == 'contentBlockDelta':
                    data = json.loads(payload_str)
                    delta = data.get('delta', {})
                    text = delta.get('text', '')
                    print(text)
                elif event_type == 'contentBlockStop':
                    # End of a content block
                    pass
                elif event_type == 'messageStop':
                    # End of a message
                    pass
                elif event_type == 'metadata':
                    # Process metadata if needed
                    data = json.loads(payload_str)
                    # For example, print usage metrics
                    usage = data.get('usage', {})
                    if usage:
                        print(f"\nUsage: {usage}")
                elif event_type == 'error':
                    # Handle errors
                    error_code = headers.get(':error-code')
                    error_message = headers.get(':error-message')
                    print(f"\nError {error_code}: {error_message}")
                    return
                else:
                    # Handle other event types
                    pass

if __name__ == '__main__':
    main()


# 5. Build payload

In [ ]:
def parse_message(input_data):
    # If input_data is a string
    if isinstance(input_data, str):
        return [
            {
                "content": [
                    {"text": input_data}
                ],
                "role": "user"
            }
        ]
    # If input_data is a list (assuming it's an OpenAI messages format)
    elif isinstance(input_data, list):
        content_list = []
        for message in input_data:
            if message.get("role") in ["user", "assistant"]:
                content_list.append({"text": message.get("content", "")})
        return [
            {
                "content": content_list,
                "role": "user"
            }
        ]
    else:
        raise ValueError("Invalid input: input_data must be a string or a list of messages.")


In [ ]:
# Example with a string input
string_input = "Hello how are you? Can you write 3 paragraphs about dinosaurs? Make them about anything related to them!"
parsed_message = parse_message(string_input)
print(parsed_message)
# Example with OpenAI message format
openai_messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Find the weather in New York."},
    {"role": "assistant", "content": "The weather in New York is currently cloudy with a temperature of 15°C."}
]
parsed_message = parse_message(openai_messages)
print(parsed_message)


# 6. LLMstudio tests

### 6.1 OpenAI stream example

In [ ]:
from llmstudio import LLM

In [ ]:
llm = LLM('openai/gpt-3.5-turbo')
llm.chat('Hello! hwo are you?')

### 6.2 Bedrock

In [1]:
from llmstudio import LLM

Running LLMstudio Engine on http://localhost:50001 
Running LLMstudio Tracking on http://localhost:50002 


In [2]:
llm = LLM('bedrock/amazon.titan-text-lite-v1')

In [3]:
llm.chat('Can you give me 10 paragrahps about Trees? They are supper cool!')

start chunk: ChatCompletionChunk(id='6a1be62a-d98c-495c-a268-09fc7f6f3de7', choices=[Choice(delta=ChoiceDelta(content='', function_call=None, role='assistant', tool_calls=None, refusal=None), finish_reason=None, index=0, logprobs=None)], created=1726762182, model='amazon.titan-text-lite-v1', object='chat.completion.chunk', system_fingerprint=None, usage=None)
content chunk: ChatCompletionChunk(id='476ef067-fc1f-477f-aadb-efa2c679fd8f', choices=[Choice(delta=ChoiceDelta(content='\nBased on the provided content, here is a text that includes 10 paragraphs about trees:\n\nTrees are important to our planet, providing oxygen, shelter, and beauty. They absorb carbon dioxide and release ', function_call=None, role=None, tool_calls=None), finish_reason=None, index=0, logprobs=None)], created=1726762182, model='amazon.titan-text-lite-v1', object='chat.completion.chunk', system_fingerprint=None, usage=None)
content chunk: ChatCompletionChunk(id='1abb157c-0a75-469f-8272-2c25a449a8dc', choices=[Cho

ChatCompletion(id='10102d8f-83a7-48b7-89a6-1002c27b4ca5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="\nBased on the provided content, here is a text that includes 10 paragraphs about trees:\n\nTrees are important to our planet, providing oxygen, shelter, and beauty. They absorb carbon dioxide and release oxygen, helping to fight climate change.\nWhile you're reading this, there's a tree growing that will eventually become a big, tall tree. Its roots will find water to drink, and food to eat. You're reading where that tree grew from, and you're helping it along the way. It will eventually give you shade in the summer and let in the sun in the winter. Trees even let birds build nests and homes in their branches. Everyone can appreciate the beauty and importance of trees, and what they offer to animals, people, and the environment.", role='assistant', function_call=None, tool_calls=None))], created=1726762182, model='amazon.titan-t

In [ ]:
messages = [
    {"role": "user", "content": "Can you give me 10 paragraphs about Trees? They are super cool!"}
]
llm.chat(messages)